In [ ]:
import os
import shutil
import pickle
import sys
from time import time
import pandas as pd
import boto3
import torch

from darts.models import (
    TFTModel,
    TiDEModel,
    TSMixerModel,
    NaiveEnsembleModel,
)


import warnings

warnings.filterwarnings("ignore")

# logging
import logging

from dotenv import load_dotenv
load_dotenv()

# define log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

t0 = time()

# adding module folder to system path
# needed for running scripts as jobs
os.chdir('..')
home = os.getenv('HOME')
module_paths = [
    f'{home}/spp_weis_price_forecast/src',
    f'{home}/Documents/github/spp_weis_price_forecast/src',
]
for module_path in module_paths:
    if os.path.isdir(module_path):
        log.info('adding module path')
        sys.path.insert(0, module_path)

log.info(f'os.getcwd(): {os.getcwd()}')
log.info(f'os.listdir(): {os.listdir()}')

# from module path
import data_engineering as de
import parameters
import utils
from modeling import build_fit_tsmixerx, build_fit_tft, build_fit_tide

# will be loaded from root when deployed

# client for uploading model weights
s3 = boto3.client('s3')

# check parameters
log.info(f'FORECAST_HORIZON: {parameters.FORECAST_HORIZON}')
log.info(f'INPUT_CHUNK_LENGTH: {parameters.INPUT_CHUNK_LENGTH}')
log.info(f'MODEL_NAME: {parameters.MODEL_NAME}')

# connect to database and prepare data
print('\n' + '*' * 40, flush=True)
con = de.create_database()

In [ ]:
log.info('preparing lmp data')
lmp = de.prep_lmp(con)
lmp_df = lmp.to_pandas().rename(
    columns={
        'LMP': 'LMP_HOURLY',
        'unique_id': 'node',
        'timestamp_mst': 'time'
    }
)


log.info('preparing covariate data')
# all_df = de.prep_all_df(con)
all_df_pd = de.all_df_to_pandas(de.prep_all_df(con))
all_df_pd.info()

lmp_all, train_all, test_all, train_test_all = de.get_train_test_all(con)
con.close()

In [ ]:
all_series = de.get_series(lmp_all)
train_test_all_series = de.get_series(train_test_all)
train_series = de.get_series(train_all)
test_series = de.get_series(test_all)

futr_cov = de.get_futr_cov(all_df_pd)
past_cov = de.get_past_cov(all_df_pd)

print('\n' + '*' * 40, flush=True)

In [ ]:
# build pretrained models
models_tsmixer = []
if parameters.USE_TSMIXER:
    for i, param in enumerate(parameters.TSMIXER_PARAMS[:parameters.TOP_N]):
        print(f'\ni: {i} \t' + '*' * 25, flush=True)
        model_tsmixer = build_fit_tsmixerx(
            series=train_test_all_series,
            val_series=test_series,
            future_covariates=futr_cov,
            past_covariates=past_cov,
            **param
        )
        models_tsmixer += [model_tsmixer]

In [ ]:
models_tide = []
if parameters.USE_TIDE:
    for i, param in enumerate(parameters.TIDE_PARAMS[:parameters.TOP_N]):
        print(f'\ni: {i} \t' + '*' * 25, flush=True)
        model_tide = build_fit_tide(
            series=train_test_all_series,
            val_series=test_series,
            future_covariates=futr_cov,
            past_covariates=past_cov,
            **param
        )
        models_tide += [model_tide]

In [ ]:
models_tft = []
if parameters.USE_TFT:
    for i, param in enumerate(parameters.TFT_PARAMS[:parameters.TOP_N]):
        print(f'\ni: {i} \t' + '*' * 25, flush=True)
        model_tft = build_fit_tft(
            series=train_test_all_series,
            val_series=test_series,
            future_covariates=futr_cov,
            past_covariates=past_cov,
            **param
        )
        models_tft += [model_tft]

## Save and upload models

In [ ]:
# create directory to store artifacts
if os.path.isdir('saved_models'):
    shutil.rmtree('saved_models')
os.mkdir('saved_models')

In [ ]:
model_timestamp = '/'.join(['saved_models', 'TRAIN_TIMESTAMP.pkl'])
with open(model_timestamp, 'wb') as handle:
    pickle.dump(pd.Timestamp.utcnow(), handle)

for i, m in enumerate(models_tide):
    m.save(f'saved_models/tide_{i}.pt')

for i, m in enumerate(models_tsmixer):
    m.save(f'saved_models/tsmixer_{i}.pt')

for i, m in enumerate(models_tft):
    m.save(f'saved_models/tft_{i}.pt')

In [ ]:
ckpt_uploads = [f for f in os.listdir('saved_models') if '.pt' in f or '.ckpt' in f or 'TRAIN_TIMESTAMP.pkl' in f]
log.info(f'ckpt_uploads: {ckpt_uploads}')

In [ ]:
# upload artifacts
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")
AWS_S3_FOLDER = os.getenv("AWS_S3_FOLDER")
for ckpt in ckpt_uploads:
    log.info(f'uploading: {ckpt}')
    s3.upload_file(f'saved_models/{ckpt}', AWS_S3_BUCKET, AWS_S3_FOLDER+f'S3_models/{ckpt}')


In [ ]:
loaded_models = utils.get_loaded_models()
log.info(f'loaded_models: {loaded_models}')

In [ ]:
models_to_delete = [l for l in loaded_models if l.split('/')[-1] not in ckpt_uploads]
log.info(f'models_to_delete: {models_to_delete}')

In [ ]:
for del_model in models_to_delete:
    log.info(f'removing: {del_model}')
    s3.delete_object(Bucket=AWS_S3_BUCKET, Key=del_model)

## Test loading models from S3 and doing inference

In [ ]:
local_dir = 's3_models/'
if os.path.isdir(local_dir):
    shutil.rmtree(local_dir)
os.mkdir(local_dir)

In [ ]:
loaded_models = utils.get_loaded_models()
log.info(f'loaded_models: {loaded_models}')

In [ ]:
# lm.split('/')[-1]

In [ ]:
for lm in loaded_models:
    local_file = local_dir +lm.split('/')[-1]
    log.info(f'downloading: {lm} to {local_file}')
    s3.download_file(Bucket=AWS_S3_BUCKET, Key=lm, Filename=local_file)

### Load models by type

In [ ]:
ts_mixer_ckpts = [f for f in os.listdir(local_dir) if 'tsmixer' in f and '.pt' in f and '.ckpt' not in f and 'TRAIN_TIMESTAMP.pkl' not in f]
ts_mixer_ckpts

In [ ]:
ts_mixer_forecasting_models = []
for m_ckpt in ts_mixer_ckpts:
    log.info(f'loading model: {m_ckpt}')
    ts_mixer_forecasting_models += [TSMixerModel.load(f's3_models/{m_ckpt}', map_location=torch.device('cpu'))]

In [ ]:
tide_ckpts = [f for f in os.listdir(local_dir) if 'tide_' in f and '.pt' in f and '.ckpt' not in f and 'TRAIN_TIMESTAMP.pkl' not in f]
tide_ckpts

In [ ]:
tide_forecasting_models = []
for m_ckpt in tide_ckpts:
    log.info(f'loading model: {m_ckpt}')
    tide_forecasting_models += [TiDEModel.load(f's3_models/{m_ckpt}', map_location=torch.device('cpu'))]

In [ ]:
tft_ckpts = [f for f in os.listdir(local_dir) if 'tft' in f and '.pt' in f and '.ckpt' not in f and 'TRAIN_TIMESTAMP.pkl' not in f]
tft_ckpts

In [ ]:
tft_forecasting_models = []
for m_ckpt in tft_ckpts:
    log.info(f'loading model: {m_ckpt}')
    tide_forecasting_models += [TFTModel.load(f's3_models/{m_ckpt}', map_location=torch.device('cpu'))]

## Create ensemble model

In [ ]:
forecasting_models = ts_mixer_forecasting_models + tide_forecasting_models + tft_forecasting_models

In [ ]:
logging.basicConfig(level=logging.INFO)

# test predictions on latest run
print('\n' + '*' * 40, flush=True)
log.info('loading model from checkpoints')
loaded_model = NaiveEnsembleModel(
        forecasting_models=forecasting_models, 
        train_forecasting_models=False
    )

log.info('test getting predictions')
plot_ind = 3
plot_series = all_series[plot_ind]

plot_end_time = plot_series.end_time() - pd.Timedelta(f'{parameters.INPUT_CHUNK_LENGTH + 1}h')
log.info(f'plot_end_time: {plot_end_time}')

plot_node_name = plot_series.static_covariates.unique_id.LMP
node_series = plot_series.drop_after(plot_end_time)
log.info(f'plot_end_time: {plot_end_time}')
log.info(f'node_series.end_time(): {node_series.end_time()}')
future_cov_series = futr_cov[0]
past_cov_series = past_cov[0]

data = {
    'series': [node_series.to_json()],
    'past_covariates': [past_cov_series.to_json()],
    'future_covariates': [future_cov_series.to_json()],
    'n': 5,
    'num_samples': 2
}
df = pd.DataFrame(data)

df['num_samples'] = 2
pred=loaded_model.predict(
    series=node_series,
    past_covariates=past_cov_series,
    future_covariates=future_cov_series,
    n=5,
    num_samples=2,
)

print('\n' + '*' * 40, flush=True)
log.info(f'pred: {pred}')

In [ ]:
pred.pd_dataframe()

In [ ]:
t1 = time()
log.info('finished retraining')
log.info(f'total time (min): {(t1-t0)/60:.2f}')